In [69]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
import re
import string 
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fachr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
train = pd.read_csv('Dataset_Capstone_Clean.csv')

train.head()

,Text,Mood
0,i did there was a mental debate i can eat the ...,anger
1,i dont know how but i forgot how to communicat...,anger
2,i started to feel grumpy and frustrated,anger
3,i also feel insincere shallow and fake,anger
4,i feel petty for it but i think less of you as...,anger


In [71]:
import pandas as pd

mood_mapping = {
    "anger": 0,
    "joy": 1,
    "sadness": 2,
    "happy": 3,
    "worry": 4,
    "fear": 5,
    "love": 6,
    "neutral": 7
}

# Sample data

# Apply the mapping
train['Mood'] = train['Mood'].map(mood_mapping)

print(train)

                                                    Text  Mood
0      i did there was a mental debate i can eat the ...     0
1      i dont know how but i forgot how to communicat...     0
2                i started to feel grumpy and frustrated     0
3                 i also feel insincere shallow and fake     0
4      i feel petty for it but i think less of you as...     0
...                                                  ...   ...
39995  JamieLynnMB i had my son when i was 16hes now ...     4
39996                      sneezing is never a good sign     4
39997                             Awe i feel so left out     4
39998  LAPPYTOP BATERRRY DYINGtryingtofind a movieto ...     4
39999  going crazy the pain is unbearableamp i gotta ...     4

[40000 rows x 2 columns]


In [72]:
train.Mood.value_counts()

Mood
0    5000
5    5000
3    5000
1    5000
6    5000
7    5000
2    5000
4    5000
Name: count, dtype: int64

In [73]:
print("Duplicate entries in the dataset: " + str(train.duplicated().sum()))

Duplicate entries in the dataset: 16


In [74]:
train = train.drop_duplicates()

In [75]:
print("Duplicate entries in the dataset: " + str(train.duplicated().sum()))

Duplicate entries in the dataset: 0


In [76]:
stopwords_english = set(stopwords.words('english'))
my_stopwords = set(["http", "s", "nt", "'m", "re", "havent", "'ve", "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "hed", "hell", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "it's", "its", "itself", "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "shell", "shes", "should", "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "there", "theres", "these", "they", "theyd", "they'll", "theyre", "theyve", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "wed", "we'll", "were", "weve", "were", "what", "what's", "when", "whens", "where", "where's", "which", "while", "who", "whos", "whom", "why", "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself", "yourselves" ])
stopwords_english.update(my_stopwords)

def preprocess_review(text):
    text = text.lower()

    text = re.sub(r'\d+', '', text)

    text = re.sub(r'[^\w\s]', '', text)

    tokens = text.split()

    clean_tokens = [tok for tok in tokens if tok not in stopwords_english and len(tok) > 1]
    

    clean_text = ' '.join(clean_tokens)

    return clean_text

In [77]:
sample = train['Text'][100]

print('ORIGINAL REVIEW:   ' + sample + '\n')

print('WITH PROCESSING:    ' + preprocess_review(sample))

ORIGINAL REVIEW:   i hate to admit it but im feeling envious of bob because hes just been complimented for his work and i havent

WITH PROCESSING:    hate admit feeling envious bob complimented work


In [78]:
train["Text"] = train["Text"].apply(preprocess_review)

In [55]:
X_train = train["Text"].values
y_train = train["Mood"].values

In [56]:
X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [57]:
print("Training data:", X_tr.shape, y_tr.shape)
print("Validation data:", X_va.shape, y_va.shape)

Training data: (31987,) (31987,)
Validation data: (7997,) (7997,)


In [58]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X_tr)

max_seq_length = 250

X_tr_seq = tokenizer.texts_to_sequences(X_tr)
X_tr_seq = pad_sequences(X_tr_seq, maxlen=max_seq_length)

X_va_seq = tokenizer.texts_to_sequences(X_va)
X_va_seq = pad_sequences(X_va_seq, maxlen=max_seq_length)

In [95]:
model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=128, input_length=250))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(units=128)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(units=32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(units=8, activation='softmax'))

In [96]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [97]:
history = model.fit(X_tr_seq,
                    y_tr,
                    validation_data=(X_va_seq, y_va),
                    batch_size=64,
                    epochs=15,
                    verbose=1,
                   )

Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 280s 513ms/step - acc: 0.2005 - loss: 2.4072 - val_acc: 0.1387 - val_loss: 2.2884
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 252s 503ms/step - acc: 0.3820 - loss: 1.6459 - val_acc: 0.4622 - val_loss: 1.4687
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 251s 502ms/step - acc: 0.5884 - loss: 1.0791 - val_acc: 0.6179 - val_loss: 0.9523
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 247s 493ms/step - acc: 0.6797 - loss: 0.8025 - val_acc: 0.6620 - val_loss: 0.8364
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 246s 492ms/step - acc: 0.7120 - loss: 0.6846 - val_acc: 0.6515 - val_loss: 0.9185
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 239s 478ms/step - acc: 0.7404 - loss: 0.5980 - val_acc: 0.6637 - val_loss: 0.8780
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 234s 468ms/step - acc: 0.7570 - loss: 0.5579 - val_acc: 0.6430 - val_loss: 0.8697
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 239s 477ms/step - acc: 0.7785 - loss: 0.5093 - val_acc: 0.6776 - val_loss: 0.8272
Epoch 9/15
500/500 ━━━━━